## Titanic Survival : Predicitve Model Notebook [Using Random Forest]
**Author**:🧕🏿 Hasanat Owoseni \
**Date** : 14th October, 2019

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
test_df = pd.read_csv('cleaned_test.csv')
train_df = pd.read_csv('cleaned_train.csv')

In [3]:
test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 21 columns):
employeeno                                                38312 non-null object
trainings_attended                                        38312 non-null int64
last_performance_score                                    38312 non-null float64
targets_met                                               38312 non-null int64
previous_award                                            38312 non-null int64
training_score_average                                    38312 non-null int64
no_of_previous_employers                                  38312 non-null int64
promoted_or_not                                           38312 non-null int64
age                                                       38312 non-null int64
working_for                                               38312 non-null int64
is_female                                                 38312 non-null int64
schooled_abroad    

In [4]:
columns = [column for column in test_df.columns if column != 'employeeno']
test_x = test_df[columns].values.astype('float')

test_empid = test_df['employeeno']

In [5]:
columns = [column for column in train_df.columns if column != ('promoted_or_not' ) and column !=( 'employeeno')]

X = train_df[columns].values.astype('float')
s_train = train_df[columns]

y = train_df['promoted_or_not']
empid = train_df['employeeno']

In [6]:
print(test_x.shape)
print(X.shape)
s_train.info()

(16496, 19)
(38312, 19)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
trainings_attended                                        38312 non-null int64
last_performance_score                                    38312 non-null float64
targets_met                                               38312 non-null int64
previous_award                                            38312 non-null int64
training_score_average                                    38312 non-null int64
no_of_previous_employers                                  38312 non-null int64
age                                                       38312 non-null int64
working_for                                               38312 non-null int64
is_female                                                 38312 non-null int64
schooled_abroad                                           38312 non-null int64
is_married                                                38312 non-null in

In [7]:
from sklearn.model_selection import train_test_split
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [8]:
# random forest model creation
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X_train,y_train)

# predictions
rfc_predict = rfc.predict(test_x)

C:\Users\NOTEBOOK\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
xrcc =  rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test,xrcc ))
print('\n')

print("=== Classification Report ===")
print(classification_report(y_test, xrcc))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[11458   144]
 [  885   156]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     11602
           1       0.52      0.15      0.23      1041

    accuracy                           0.92     12643
   macro avg       0.72      0.57      0.59     12643
weighted avg       0.89      0.92      0.90     12643



=== All AUC Scores ===
[0.73045478 0.71822097 0.72975874 0.75225695 0.7255687  0.75614996
 0.7440265  0.74445025 0.74331584 0.76111622]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7405318901232063


In [ ]:
df_submission = pd.DataFrame({'employeeno':test_df['employeeno'], 'promoted_or_not':rfc_predict})
df_submission.to_csv('randomforest.csv', index = False)

----------------------------------

In [19]:
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }


In [21]:
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [22]:
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 17.1min finished


{'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': None}


In [23]:
rfc = RandomForestClassifier(n_estimators=600, max_depth=300, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(test_x)

In [24]:
df_submission = pd.DataFrame({'employeeno':test_df['employeeno'], 'promoted_or_not':rfc_predict})
df_submission.to_csv('rando.csv', index = False)

In [32]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=10000, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(test_x)
df_submission = pd.DataFrame({'employeeno':test_df['employeeno'], 'promoted_or_not':rfc_predict})
df_submission.to_csv('rando2.csv', index = False)